## Description
Notes covering the logic behind the ADAM optimizer.

## Stochastic Gradient Descent (SGD)

Speedier than ordinary gradient descent. In principal, SGD trains on subsets of our sample dataset at each step. Which we already do with Minibatches.

## Gradient Descent with Momentum

Basic idea is to compute an exponentially weighted average of the gradients and use that moving average to update the gradients.

Basic formulation:    

Compute gradients for weights (dw) and biases (db) for current minibatch w.r.t. the loss function. 

Then compute Vdw = beta * Vdw + (1 - beta)*dw | Vdb = beta * Vdb + (1 - beta)*dw

Then compute w = w - lr * Vdw | b = b - lr * Vdb

These look like simple moving averages but they are actually exponential moving averages because the weights and biases for each of the previous values decay exponentially.

This process smoothes out the steps of gradient descent. It reduces extreme movements in the direction where the loss function has the most extreme movements from one minibatch to another. This can allow us to use a larger learning rate more safely.

A common analogy for the gradient terms and update terms is acceleration and velocity. The gradients (dw & db) are like acceleration, they increase the velocity of the gradient descent. The velocity is the update terms (Vdw & Vdb), they are the speed at which the model's parameters (w & b) are updated. In this formulation, the betas can be though of as friction that moderates the acceleration of the gradient descent.

In this basic formulation, our parameters are w & b and our hyperparameters are lr & beta. The 'velocity' terms (Vdw and Vdb) need to be initialized, typically at zero-valued vectors.

## RMSProp
#### Root Mean-Squared Prop

For this explaination, consider parameters w & b not as weights and biases but as directions (horizontal = w, vertical = b) in which the loss function can move. At each iteration, we comput the gradients dw & db w.r.t. the loss function for the current minibatch just like normal. 

As with momentum-based descent, we calculate exponential moving averages used to update each direction: Sdw = beta * Sdw + (1 - beta)*dw**2 | Sdb = beta * Sdb + (1 - beta)*db**2

Unlike momentum-based descent, the gradients used in the moving average calculations are squared. NOTE: these are elementwise square operations, so each element of the gradient tensors is squared. 

Then we update the parameters as: w = w - lr * (dw / sqrt(Sdw)) | b = b - lr * (db / sqrt(Sdb))

The benefit of RSMProp is once again in smoothing out the iteration-over-iteration change in the loss function. However, it may improve upon ordinary momentum-based gradient descent by smoothing more in one direction than another. The direction in which the loss function varies the most from one minibatch to another will have the largest gradients, it is also the direction in which we would want to produce greater smoothing. Let's say this is the vertical direction b. Because it has larger gradients, an elementwise squaring of db will grow Sdw by more than the elementwise squaring of dw will grow Sdw. Therefore, the update to b (-lr * db / sqrt(Sdb)) will be smaller than the update to w. In this way, RMSProp provides greater smoothing to parameters that cause the largest inter-batch variance in the loss function.

In practice, the 'vertical' and 'horizontal' directions are sets of several parameters because we typically build and train models in high-dimensional spaces. We often also add a small epsilon term to Sdw and Sdb to ensure that we don't divide by zero.

Again, this can allow us to increase the model's learning rate.

## Adam Optimization Algorithm
### Adaptive Moment Estimation
#### Blending Momentum and RMSProp

Initialize: 

    Vdw, Sdw = 0, 0 | Vdb, Sdb = 0, 0

Compute gradients dw, db for current minibatch w.r.t. the loss function.

Now we use a beta1 hyperparameter for the momentum updates (first moment):

    Vdw = beta1 * Vdw + (1 - beta1)*dw | Vdb = beta1 * Vdb + (1 - beta1)*db
    
Then we use a different hyperparameter, beta2, for the RSMProp updates (second moment):

    Sdw = beta2 * Sdw + (1 - beta2)*dw**2 | Sdb = beta2 * Sdb + (1 - beta2)*db**2
    
Typicall, Adam implements bias corrections. Indeed, pytorch's optimize.Adam includes the corrections:

    Vdw(corrected) = Vdw / (1 - beta1**t) | Vdb(corrected) = Vdb / (1 - beta1**t)
    Sdw(corrected) = Sdw / (1 - beta2**t) | Sdb(corrected) = Sdb / (1 - beta2**t)
        Where t is the iteration of training that the model is on.
        
Then, the parameters are updated through a combination of momentum and RMSProp updates:

    w = w - lr * (Vdw(corr) / (sqrt(Sdw(corr)) + epsilon)
    b = b - lr * (Vdb(corr) / (sqrt(Sdb(corr)) + epsilon)
    
So, in this description, we are only looking at two parameters (w & b) but in application the same process is used for all parameters.

The hyperparameters are:
    
    lr, beta1, beta2, and epsilon
    Pytorch Defaults: (0.001, 0.9, 0.999, and 1e-08)